<a href="https://colab.research.google.com/github/BenBritons/DS_notebooks/blob/main/NLP/MachineTranslationWordEmbadding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Multilingual Embedding-based Machine Translation


**In this notebook** **<font color='red'>I</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully).

For our system we choose two kindred Slavic languages: Ukranian and Russian.

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings


In [2]:
!curl -o cc.ru.300.vec.gz https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
!gunzip cc.ru.300.vec.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1245M  100 1245M    0     0  19.3M      0  0:01:04  0:01:04 --:--:-- 20.7M


In [3]:
!curl -o cc.uk.300.vec.gz https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
!gunzip cc.uk.300.vec.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1199M  100 1199M    0     0  19.4M      0  0:01:01  0:01:01 --:--:-- 22.9M


In [4]:
#!pip install fasttext

In [5]:
#!pip install gensim


Load embeddings for ukranian and russian.

In [6]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [7]:
len(uk_emb)

2000000

In [8]:
ru_emb = KeyedVectors.load_word2vec_format('cc.ru.300.vec')

In [9]:
ru_emb.most_similar([ru_emb["дом"]], topn=10)

[('дом', 1.0),
 ('особняк', 0.7073208689689636),
 ('коттедж', 0.697927713394165),
 ('домик', 0.6890208125114441),
 ('Дом', 0.6677538752555847),
 ('дом.В', 0.6623372435569763),
 ('дом.А', 0.6590256094932556),
 ('дом-', 0.6414237022399902),
 ('загородный', 0.6390586495399475),
 ('офис', 0.6238341331481934)]

In [10]:
uk_emb.most_similar([uk_emb["хата"]])

[('хата', 1.0000001192092896),
 ('хата-', 0.7466036677360535),
 ('хатка', 0.6733182668685913),
 ('хатина', 0.6623023152351379),
 ('хатинка', 0.6520452499389648),
 ('хата-мазанка', 0.64560467004776),
 ('reakcji', 0.6153016686439514),
 ('оселя', 0.6063539385795593),
 ('курна', 0.60240638256073),
 ('Хата', 0.5938948392868042)]

In [11]:
ru_emb.most_similar([uk_emb["хата"]])

[('-Александр', 0.2608003318309784),
 ('сopeвнoваниях', 0.2597772479057312),
 ('подрядиться', 0.259461373090744),
 ('дрожках', 0.25474271178245544),
 ('плясун', 0.25230672955513),
 ('Фёдорыч', 0.24858714640140533),
 ('вызвалась', 0.2417260855436325),
 ('Гедеван', 0.2410685271024704),
 ('Лихов', 0.23922650516033173),
 ('Тёркин', 0.23871248960494995)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [12]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [13]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [14]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [16]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, Y_train)

LinearRegression()

In [19]:
lr.score(X_train, Y_train)

0.5387453182044396

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [21]:
august = lr.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531403541564941),
 ('июнь', 0.8402307629585266),
 ('март', 0.8385774493217468),
 ('сентябрь', 0.8331867456436157),
 ('февраль', 0.8311492800712585),
 ('октябрь', 0.8278172016143799),
 ('ноябрь', 0.8244150876998901),
 ('июль', 0.8228995203971863),
 ('август', 0.8112361431121826),
 ('январь', 0.8022859692573547)]

In [49]:
ru_emb.most_similar(lr.predict(uk_emb['серпень'].reshape(1, -1)), topn = 9)

('апрель', 0.8531403541564941)

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [51]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
      for j in ru_emb.most_similar(lr.predict(uk_emb[_].reshape(1, -1)), topn = topn):
        if ru in j:
          num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val


In [52]:
precision([("серпень", "август")], august, topn=9)

1.0

In [53]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [62]:
print(precision(uk_ru_test, X_test))
print(precision(uk_ru_test, Y_test))
print(precision(uk_ru_test, lr.predict(X_test), 1))
print(precision(uk_ru_test, lr.predict(X_test), 5))

0.6259541984732825
0.6259541984732825
0.6259541984732825
0.7913486005089059


In [57]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

AssertionError: ignored

In [58]:
precision_top1 = precision(uk_ru_test, lr.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, lr.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

AssertionError: ignored

## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal.
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [78]:
import numpy as np
def learn_transform(X_train, Y_train):
    """
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    X = np.array(X_train)
    Y = np.array(Y_train)
    U, W, Vh = np.linalg.svd(np.transpose(X) @ Y)

    return U @ Vh

In [79]:
W = learn_transform(X_train, Y_train)

array([[ 1.0000002e+00, -1.0445934e-08,  8.2780481e-08, ...,
         1.0242603e-08, -7.0971784e-09, -2.1691626e-08],
       [-1.0445934e-08,  9.9999982e-01,  1.0376747e-08, ...,
         2.3747004e-08,  1.9949768e-08, -1.1788722e-08],
       [ 8.2780481e-08,  1.0376747e-08,  9.9999970e-01, ...,
         6.7716904e-08,  2.8304335e-08, -4.4140510e-08],
       ...,
       [ 1.0242603e-08,  2.3747004e-08,  6.7716904e-08, ...,
         9.9999946e-01, -1.6319246e-08, -3.4254217e-08],
       [-7.0971784e-09,  1.9949768e-08,  2.8304335e-08, ...,
        -1.6319246e-08,  9.9999982e-01, -9.6067403e-09],
       [-2.1691626e-08, -1.1788722e-08, -4.4140510e-08, ...,
        -3.4254217e-08, -9.6067403e-09,  1.0000001e+00]], dtype=float32)

In [86]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245132565498352),
 ('июнь', 0.8056628704071045),
 ('сентябрь', 0.8055762648582458),
 ('март', 0.8032935261726379),
 ('октябрь', 0.798710286617279),
 ('июль', 0.794679582118988),
 ('ноябрь', 0.7939637899398804),
 ('август', 0.7938188314437866),
 ('февраль', 0.7923861742019653),
 ('декабрь', 0.7715376019477844)]

In [89]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6259541984732825
0.7913486005089059


## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [ ]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [104]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translation = ""
    for word in sentence.split():
      translation += ru_emb.most_similar(lr.predict(uk_emb[word].reshape(1, -1)))[0][0] + " "

    return translation

In [105]:
translate("1 , 3")

'до , до '

In [106]:
translate("кіт зловив мишу")

'кот поймал мышка '

In [111]:
translate("кіт зловив щось")

'кот поймал что-то '

In [112]:
assert translate("кіт зловив щось") == "кот поймал что-то "

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.